# Connect to local MongoDB instance (running via Docker) and install the libraries


In [ ]:
!pip install  pandas as pd
!pip install tabulate

In [51]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")

try:
    client.admin.command("ping")
    print(" Connected Successfully")
except Exception as e:
    print("Failed to Connect", e)

# Λίστα βάσεων δεδομένων
print(client.list_database_names())



 Connected Successfully
['admin', 'config', 'lab', 'local']


# Select database and collection


In [52]:
db = client["lab"]
collection = db["restaurants"]

In [53]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)




## Question 1:
### Top 5 vegan-friendly breakfast restaurants in Athens with more than 100 reviews (by avg_rating)

In [54]:
pipeline_1 = [
    {"$match": {
        "city": "Athens",
        "vegan_options": "Y",
        "meals": {"$in": ["Breakfast"]},
        "total_reviews_count": {"$gt": 100}
    }},
    {"$sort": {"avg_rating": -1, "total_reviews_count": -1}},
    {"$project": {
        "_id": 0,
        "restaurant_name": 1,
        "address": 1,
        "avg_rating": 1,
        "total_reviews_count": 1
    }},
    {"$limit": 5}
]
results_1 = list(collection.aggregate(pipeline_1))
df1 = pd.DataFrame(results_1)
print("\n Top 5 Vegan Breakfast Restaurants in Athens\n")
print(df1.to_markdown(index=False))





 Top 5 Vegan Breakfast Restaurants in Athens

| restaurant_name   | address                                             |   avg_rating |   total_reviews_count |
|:------------------|:----------------------------------------------------|-------------:|----------------------:|
| Vegan Beat        | Perikleous 56 Ground floor, Athens 10560 Greece     |            5 |                   754 |
| Victory Cafe      | Fillelinon 22, Athens 10557 Greece                  |            5 |                   562 |
| Montakioy         | Stadiou 30 Stoa Korai, Athens 10564 Greece          |            5 |                   208 |
| Coffee Joint      | Vourvachi 5-9 Iosif ton Rogon, Athens 117 43 Greece |            5 |                   184 |
| Candy Cat         | Persefonis 59, Athens 11854 Greece                  |            5 |                   113 |


###  Question 2: Top 10 Popular Cuisines in Vegan Restaurants in Athens

Identify the **10 most popular cuisine types** (field: `cuisines`) in **Athens** that are offered by restaurants with **vegan options**.

For each cuisine type, display:
- the **name of the cuisine**
- the **number of restaurants** offering it
- the **average number of total reviews** for those restaurants

Sort the results by **popularity** (number of restaurants offering each cuisine) in **descending order**.


In [55]:
pipeline_2 = [
    {"$match": {
        "city": "Athens",
        "vegan_options": "Y"
    }},
    {"$unwind": "$cuisines"},
    {"$group": {
        "_id": "$cuisines",
        "restaurant_count": {"$sum": 1},
        "avg_reviews": {"$avg": "$total_reviews_count"}
    }},
    {"$sort": {"restaurant_count": -1}},
    {"$limit": 10},
    {"$project": {
        "_id": 0,
        "cuisine": "$_id",
        "restaurant_count": 1,
        "avg_reviews": {"$round": ["$avg_reviews", 1]}
    }}
]
results_2 = list(collection.aggregate(pipeline_2))
df2 = pd.DataFrame(results_2)
print("\nTop 10 Popular Cuisines in Vegan Restaurants in Athens\n")
print(df2.to_markdown(index=False))


Top 10 Popular Cuisines in Vegan Restaurants in Athens

|   restaurant_count | cuisine       |   avg_reviews |
|-------------------:|:--------------|--------------:|
|                361 | Greek         |         433.1 |
|                349 | Mediterranean |         407.2 |
|                186 | European      |         416.4 |
|                 96 | Cafe          |         204.5 |
|                 59 | Healthy       |         674.4 |
|                 43 | Asian         |         278.9 |
|                 41 | Italian       |         197.7 |
|                 37 | Fast food     |         173.8 |
|                 36 | Bar           |         288.1 |
|                 34 | Seafood       |         431.9 |


###  Question 3: Top 5 Cities Outside Greece Offering Greek Cuisine

Find the **top 5 cities outside of Greece** with the **highest number of restaurants** that serve **Greek cuisine**.

For each city, display:
- the **name of the city**
- the **number of restaurants** offering Greek cuisine in that city

Sort the results in **descending order** based on the number of restaurants.

In [56]:
pipeline_3 = [
    {"$match": {
        "cuisines": "Greek",
        "country": {"$ne": "Greece"},
        "city": {"$ne": None}
    }},
    {"$group": {
        "_id": "$city",
        "restaurant_count": {"$sum": 1}
    }},
    {"$sort": {"restaurant_count": -1}},
    {"$limit": 5},
    {"$project": {
        "_id": 0,
        "city": "$_id",
        "restaurant_count": 1
    }}
]
results_3 = list(collection.aggregate(pipeline_3))
df3 = pd.DataFrame(results_3)
print("\n### Question 3: Top 5 Cities Outside Greece with Greek Cuisine\n")
print(df3.to_markdown(index=False))


### Question 3: Top 5 Cities Outside Greece with Greek Cuisine

|   restaurant_count | city       |
|-------------------:|:-----------|
|                119 | Munich     |
|                105 | Paris      |
|                 71 | Nuremberg  |
|                 65 | Dusseldorf |
|                 64 | Vienna     |


### Question 4: City with the Most High-Rated Restaurants

Find the **city** (regardless of country) with the **highest number of restaurants** that meet the following criteria:
- **Average rating** (`avg_rating`) greater than **4.5**
- **Total number of reviews** (`total_reviews_count`) greater than **1000**

For the result, display:
- the **name of the city**
- the **number of restaurants** that meet the criteria

Sort the cities in **descending order** and return only the **top 1**.


In [59]:
pipeline_4 = [
    {"$match": {
        "avg_rating": {"$gt": 4.5},
        "total_reviews_count": {"$gt": 1000},
        "city": {"$ne": None}
    }},
    {"$group": {
        "_id": "$city",
        "restaurant_count": {"$sum": 1}
    }},
    {"$sort": {"restaurant_count": -1}},
    {"$limit": 1},
    {"$project": {
        "_id": 0,
        "city": "$_id",
        "restaurant_count": 1
    }}
]
results_4 = list(collection.aggregate(pipeline_4))
df4 = pd.DataFrame(results_4)
print("\n City with Most High-Rated Restaurants\n")
print(df4.to_markdown(index=False))



 City with Most High-Rated Restaurants

|   restaurant_count | city   |
|-------------------:|:-------|
|                 10 | Rome   |


###  Question 5: Greek Regions with High-Rated Restaurants

Display all **regions in Greece** (field: `region`) along with the **number of restaurants** in each region that have:
- an **average rating** (`avg_rating`) greater than **4.5**

For each region, display:
- the **name of the region**
- the **number of restaurants** that meet the criteria

Sort the regions in **descending order** based on the number of qualifying restaurants.


In [60]:
pipeline_5 = [
    {"$match": {
        "country": "Greece",
        "avg_rating": {"$gt": 4.5},
        "region": {"$ne": None}
    }},
    {"$group": {
        "_id": "$region",
        "restaurant_count": {"$sum": 1}
    }},
    {"$sort": {"restaurant_count": -1}},
    {"$project": {
        "_id": 0,
        "region": "$_id",
        "restaurant_count": 1
    }}
]
results_5 = list(collection.aggregate(pipeline_5))
df5 = pd.DataFrame(results_5)
print("\nGreek Regions with High-Rated Restaurants (>4.5)\n")
print(df5.to_markdown(index=False))


Greek Regions with High-Rated Restaurants (>4.5)

|   restaurant_count | region                    |
|-------------------:|:--------------------------|
|               1328 | Attica                    |
|               1083 | South Aegean              |
|               1067 | Crete                     |
|                802 | Central Macedonia         |
|                658 | Ionian Islands            |
|                476 | Peloponnese               |
|                337 | Northeast Aegean Islands  |
|                265 | Thessaly                  |
|                240 | Central Greece            |
|                231 | Epirus                    |
|                224 | West Greece               |
|                173 | East Macedonia and Thrace |
|                 93 | West Macedonia            |
|                 53 | Sporades                  |


###  Query Optimization: Using Indexes

To improve the performance of the aggregation queries in Question 2 (and similar ones), we can create **appropriate indexes** on fields that are frequently used in filtering stages (`$match`) and have high cardinality.

####  Why use indexes?
- Queries that filter large collections using non-indexed fields will require scanning **every document** (full collection scan).
- By indexing key fields, MongoDB can **quickly locate matching documents**, significantly reducing execution time and resource usage.

####  How we optimized the queries:

In MongoDB, indexes are crucial for optimizing query performance — especially when dealing with large datasets. Fields that are frequently used in filtering conditions (e.g., in the `$match` stage of an aggregation pipeline) and have **high cardinality** (many unique values) are typically **ideal candidates** for indexing.

####  When to Use Indexes

Indexes are recommended when:
- A field is **frequently queried**, especially in `$match`, `$sort`, or `$group`
- The field has **high cardinality** (e.g., `city`, `restaurant_name`) — meaning many unique values
- The field is part of **compound filters** (e.g., `{ city: "Athens", vegan_options: "Y" }`)
- You want to improve performance in **read-heavy workloads**
####  Drawbacks of Excessive or Improper Indexing

-  **Memory Usage**: Indexes consume RAM. Creating too many indexes may lead to increased memory usage, especially on large collections.

-  **Slower Writes**: Each `insert`, `update`, or `delete` operation must also update the relevant indexes. This adds overhead and can slow down write-heavy workloads.

-  **Maintenance Cost**: Indexes need to be maintained during operations like replication, backups, and migrations. This can increase the complexity and cost of managing the database.

-  **Low Benefit on Low-Cardinality Fields**: Indexing fields with very few unique values (e.g., `vegan_options` with only "Y" or "N") provides minimal performance improvement, and might even waste resources.
####  Best Practices for Index Usage

-  **Measure before and after**: Always evaluate query performance using `.explain("executionStats")` to justify the index.
-  **Use compound indexes carefully**: The **order of fields** in a compound index matters. It should match the order in your query filters.
-  **Index selective fields**: Prefer indexing fields that help **filter out a large portion of the dataset**, reducing scan time.
-  **Remove unused indexes**: Regularly review and **clean up redundant or unused indexes** to reduce memory usage and maintenance cost.




In [61]:

# Run explain on aggregation
explain_result = db.command({
    "explain": {
        "aggregate": "restaurants",
        "pipeline": pipeline_1,
        "cursor": {}
    },
    "verbosity": "executionStats"
})

# Print results
stats = explain_result["executionStats"]
plan = explain_result["queryPlanner"]["winningPlan"]
print("Aggregation Explain")
print("Execution Time (ms):", stats["executionTimeMillis"])
print("Documents Examined :", stats["totalDocsExamined"])
scan_stage = plan["inputStage"]["inputStage"]["stage"]
print(f"Scan stage used: {plan["inputStage"]["inputStage"]["stage"]}")



Aggregation Explain
Execution Time (ms): 619
Documents Examined : 1083397
Scan stage used: COLLSCAN


The query performed a full collection scan (`COLLSCAN`), which indicates that **no index was used**. As a result, the execution time is relatively high and MongoDB had to scan over 1 million documents to find the matching results. This clearly shows the need for proper indexing on the filtered fields.

Run the next cell and re-run the previous one to see the difference
If you want to remove the index at the end run the cell collection.drop_index



In [ ]:
collection.create_index([("city", 1)])

In [ ]:
collection.drop_index('city_1')
